In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def calculate_vpt(df, price_column='close_2'):
    df['Price Change'] = df[price_column].pct_change()
    df['VPT'] = (df['Price Change'] * df['volume']).cumsum()
    return df

def add_signal_line(df, window=14):
    df['Signal Line'] = df['VPT'].rolling(window=window).mean()
    return df

def add_moving_averages(df, short_window=5, long_window=20, price_column='close_2'):
    df['Short MA'] = df[price_column].rolling(window=short_window).mean()
    df['Long MA'] = df[price_column].rolling(window=long_window).mean()
    return df

def add_rsi(df, window=14, price_column='close_2'):
    delta = df[price_column].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df

def generate_trading_signals(df, vpt_threshold=0.005, volume_threshold=1e4, rsi_threshold=50, rsi_diff_threshold=1):
    df['Signal_Volume'] = np.where(
        (df['VPT'] > df['Signal Line']) &
        (df['VPT'].diff() > vpt_threshold) &
        (df['volume'] > volume_threshold) &
        (df['RSI'] > rsi_threshold) &
        (df['RSI'].diff() > rsi_diff_threshold) &
        (df['Short MA'] > df['Long MA']),
        0.5,  # Buy Signal
        np.where(
            (df['VPT'] < df['Signal Line']) &
            (df['VPT'].diff() < -vpt_threshold) &
            (df['volume'] > volume_threshold) &
            (df['RSI'] < rsi_threshold) &
            (df['RSI'].diff() < -rsi_diff_threshold) &
            (df['Short MA'] < df['Long MA']),
            -0.5,  # Sell Signal
            0   # No Signal
        )
    )
    return df

def plot_vpt(df):
    plt.figure(figsize=(14, 7))
    plt.plot(df.index, df['VPT'], label='VPT', color='blue')
    plt.plot(df.index, df['Signal Line'], label='Signal Line', color='red')
    
    # Plot signals
    buy_signals = df[df['Signal_Volume'] == 0.5]
    sell_signals = df[df['Signal_Volume'] == -0.5]
    
    plt.plot(buy_signals.index, df.loc[buy_signals.index, 'VPT'], '^', markersize=10, color='green', label='Buy Signal')
    plt.plot(sell_signals.index, df.loc[sell_signals.index, 'VPT'], 'v', markersize=10, color='red', label='Sell Signal')
    
    plt.title('Volume Price Trend (VPT) Indicator with Trading Signals (Close_2)')
    plt.xlabel('Date')
    plt.ylabel('VPT')
    plt.legend()
    plt.show()

def main(df, vpt_window=14, signal_window=14, short_window=5, long_window=20, rsi_window=14):
    df = calculate_vpt(df, price_column='close_2')
    df = add_signal_line(df, window=signal_window)
    df = add_moving_averages(df, short_window=short_window, long_window=long_window, price_column='close_2')
    df = add_rsi(df, window=rsi_window, price_column='close_2')
    df = generate_trading_signals(df)
    plot_vpt(df)


main(df)
